In [ ]:
import keras
from pathlib import Path
from matplotlib import pyplot as plt

In [ ]:
data_root_path = Path('/home/ramin/ramin_programs/files/large_files/fruits-100')

train_data_path = data_root_path / 'train'


In [ ]:
train_data = keras.utils.image_dataset_from_directory(
    train_data_path, image_size=(150, 150))

In [ ]:
train_data_iterator = train_data.as_numpy_iterator()
k1 = next(train_data_iterator)

print(k1[1][0])
plt.imshow(k1[0][0].astype('uint8'))
plt.show()

In [ ]:
model_k1 = keras.applications.MobileNet()

In [ ]:
model_k1.layers[-3].get_weights()

In [ ]:
base_model.layers[-1].name

In [ ]:
[x.name for x in model_k1.layers[-6:]]

In [ ]:
base_model = keras.applications.MobileNet(include_top=False)

In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(100, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_data, epochs=20)